# Читаем базы

In [6]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## База 1

In [ ]:
base_1 = pd.ExcelFile('/content/drive/MyDrive/Системный Блок/пишу тебе/открытки базы/Расшифровка открыток (основная база).xlsx')
base_1 = pd.read_excel(base_1, 'Открытки')
base_1['Номер открытки'] = base_1['Номер открытки'].astype(int)
base_1 = base_1[~(((base_1['Pic_url_1'].isna()) | (base_1['Pic_url_1'] == 0.0)) &
                ((base_1['Pic_url_2'].isna()) | (base_1['Pic_url_2'] == 0.0)))
                ]
base_1.head()

In [ ]:
base_1.shape

(6031, 61)

## База 2

In [ ]:
base_2 = pd.ExcelFile('/content/drive/MyDrive/Системный Блок/пишу тебе/открытки базы/Расшифровка открыток 2 (Ярмарка проектов).xlsx')
base_2 = pd.read_excel(base_2, 'Открытки')
base_2['Номер открытки'] = base_2['Номер открытки'].astype(int)
base_2.head()

In [ ]:
base_2.shape

(4968, 61)

## База 3

In [ ]:
base_3 = pd.ExcelFile('/content/drive/MyDrive/Системный Блок/пишу тебе/открытки базы/Расшифровка открыток 3 (Ярмарка проектов).xlsx')
base_3 = pd.read_excel(base_3, 'Открытки')
base_3 = base_3[~base_3['Номер открытки'].isna()]
base_3['Номер открытки'] = base_3['Номер открытки'].astype(int)
base_3 = base_3.rename(columns={'Resize_ur_1': 'Resize_url_1', 'Resize_ur_2': 'Resize_url_2'})
base_3.head()

In [ ]:
base_3.shape

(10453, 61)

## База 4

In [ ]:
base_4 = pd.ExcelFile('/content/drive/MyDrive/Системный Блок/пишу тебе/открытки базы/Расшифровка открыток 4 (ярмарка проектов).xlsx')
base_4 = pd.read_excel(base_4, 'Открытки')
base_4 = base_4[~base_4['Номер открытки'].isna()]
base_4['Номер открытки'] = base_4['Номер открытки'].astype(int)
base_4 = base_4[~(((base_4['Pic_url_1'].isna()) | (base_4['Pic_url_1'] == 0.0)) &
                ((base_4['Pic_url_2'].isna()) | (base_4['Pic_url_2'] == 0.0)))
                ]
base_4.head()

In [ ]:
base_4.shape

(9381, 61)

## Объединяем базы

In [ ]:
base_1['base'] = 1
base_2['base'] = 2
base_3['base'] = 3
base_4['base'] = 4

In [ ]:
df = pd.concat([base_1, base_2, base_3, base_4])
df.shape

(30833, 67)

In [ ]:
df['base'].value_counts()

3    10453
4     9381
1     6031
2     4968
Name: base, dtype: int64

# Фильтрация
Оставляем открытки, которые проходят фильтры

In [ ]:
df = df[
    (df['Разрешение на публикацию'] == 'Разрешаю публикацию') &
    (df['Статус (распределено, в процессе, завершено, проблема)'] == 'Проверено') &
    (df['Проблема'] == 'Отсутствует') &
    (~df['Resize_url_1'].isna()) & (~df['Resize_url_2'].isna())
]
df.shape

(17727, 67)

Оставляем только нужные столбцы

In [ ]:
df = df[[
    'Номер открытки',  # синяя секция (базовая)
    'Pic_url_1',
    'Pic_url_2',
    'Resize_url_1',
    'Resize_url_2',

    'Язык текста открытки',	 # зеленая секция (расшифровка)
    'Текст открытки',	
    'Перевод текста открытки (если на иностранном языке)',	
    'Вид орфографии',	
    'Дата написания текста (если автор указал явно, не штемпели)',
    'Адрес отправителя',
    'Имя отправителя',	
    'Адрес получателя',	
    'Индекс получателя',	
    'Имя получателя',	
    'Дата открытки со штемпеля (место отправления)',
    'Дата открытки со штемпеля (место, куда открытка прибыла)',	
    'Тип изображения (фотография, картина, другое)',
    'Описание изображения. Обычно приводится на задней стороне или лицевой. Там обычно указывается, что изображено на открытке, автор, год фотографии/картины',
    'Дата печати открытки',	
    'Описание открытки. Например, это может быть издательство, цена открытки и прочая интересная метаинформация',	
    'Доп. сведения о получателе (например, титул, "дядя", "товарищ")',
    'Доп. сведения об отправителе (например, титул, "твой дядя", "князь")',

    'Период (дореволюционный, советский, постсоветский)',	 # оранжевая секция (модерация)
    'Страна (откуда)',	
    'Населенный пункт (откуда)',
    'Страна (куда)',	
    'Населенный пункт (куда)',
    'Тег_1',	
    'Тег_2',
    'Тег_3',	
    'Тег_4',	
    'Тег_5',	
    'Тег_6',	
    'Тег_7'
    ]]

# Предобработка (валидация)

In [ ]:
import re
import numpy as np
from datetime import datetime

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_cols = list(df.columns)
df_cols = [col for col in df_cols if col not in 
           ['Номер открытки', 'Pic_url_1', 'Pic_url_2', 'Resize_url_1', 'Resize_url_2']]

## Подготовка
* копируем исходное состояние столбцов 
* создаем столбцы для логирования ошибок

In [ ]:
for col in df_cols:
  df[f'{col}_old'] = df[col]  # исходное значение
  df[f'{col}_error'] = False  # была ли ошибка
  df[f'{col}_fixed'] = True  # была ли ошибка исправлена

## [отсутствует]
* если текст/дата отсутствует, ставится [отсутствует] 
* [отсутствует] - только такое написание (без точки, опечаток, с маленькой буквы)


In [ ]:
ots = '[отсутствует]'

In [ ]:
ots_variations = [
 'Отсутствует',
 '[ отсутствует]',
 '{отсутствует]',
 '[отсутвствует]',
 '[отсутсмтвует]',
 'отсутсвует',
 'отсутствует',
 '[отcутствует]',
 '[отсуствует]',
 '[отсутствует[',
 '[отсутствут]',
 '[отсутствует ]',
 '[отсустствует]',
 '[отсутствует',
 'отсутствует',
 '[отсутствует}',
 '[отсутствуют]',
 '[отсутствует]',
 '[отстутсвует]',
 '[отсутсвует]',
 '[отсцутствует]',
 '[Отсутствует]',
 '[отсутстсвует]',
 '[отсутсвтует]',
 '[отстутствует]',
 '[отсутствтует]',
 '[отсутствие]',
 'отсутсnвует',
 '[отсутсnвует]',
 '[? отсутствует]',
 '[отсутвтует]'
 ]

In [ ]:
def do_replace(s):
  # если пусто, ставим [отсутствует]
  if pd.isna(s):
    return ots

  # иначе приводим к строковому виду, обрезаем пробелы
  s = str(s).lstrip().rstrip()

  # если [отсутствует] написано с ошибкой, исправляем
  if s == '' or s in ots_variations:
    return ots

  return s

Обрабатываем все столбцы

In [ ]:
for col in df_cols:
  df[col] = df[col].apply(do_replace)

## [нрзб]

Смотрим, какие есть вариации написания нрзб

In [ ]:
nrzb = '[нрзб]'

In [ ]:
for col in df.columns:
  values = df[col].to_list()
  for v in values:
    if not isinstance(v, str):
      continue
    if nrzb not in v and 'нрзб' in v:
      print(v)

Составляем список

In [ ]:
nrzb_variations = [
 '[нрзб.]',
 '[нрзб ]',
 '<нрзб>',
 '<нрзб.>',
 '[нрзбр]',
 '[нрзбр.]',
 '[нрзбр.,]',
 '<нрбр>',
 '[?нрзб]',
 '[нрзб[',
 '[]нрзб',
 '[ нрзб]',
 '[ нрзб.]',
 '[нрзб. ]',
 '{нрзб.}',
 '{нрзб}',
 '[ нрзб ]'
 ]

nrzb_replacement_dict = {v: nrzb for v in nrzb_variations}

nrzb_replacement_dict['нрзб] Б[...] Городская больница'] = '[нрзб] Б[...] Городская больница'
nrzb_replacement_dict['Гор. Кострома Царевская улица, дом Леонтьева, кв. № 3 (В Казарме 183 нрзб)'] = \
  'Гор. Кострома Царевская улица, дом Леонтьева, кв. № 3 (В Казарме 183 [нрзб])'
nrzb_replacement_dict['Мадемуазель Ольга нрзб'] = 'Мадемуазель Ольга [нрзб]'
nrzb_replacement_dict[
    'Доктор сказал, что мне полезно не только легкое чтение, но и какое-либо серьезное  нрзб.'
    ] = 'Доктор сказал, что мне полезно не только легкое чтение, но и какое-либо серьезное [нрзб]'
nrzb_replacement_dict['<u>нрзб]'] = '<u>[нрзб]'
nrzb_replacement_dict['[<s>нрзб</s>]'] = '<s>[нрзб]</s>'

# nrzb_variations += [
# '[нрзб.?мечеть]',
# '[нрзб. художник Г. Рожков?]',
# '[нрзб - ва]',
# '[Лит А - нрзб]',
# '[нрзб.59].',
# '[отсутствует*нрзб.]',
# '[нрзб. *?г. Петербург]',
# '[нрзб. подпись]',
# '[нрзб.?фамилия] '
# '[нрзб *Костя? *Коля?]',
# '[срочно нрзб]',
# '[П-нрзб]',
# '[Евсе-нрзб]',
# '[нрзб - ва]',
# '[Лит А - нрзб]',
# ]

Убираем вариативность [нрзб] и логируем

In [ ]:
def replace_log(row, col):
  s = row[col]
  err = row[f'{col}_error']
  fix = row[f'{col}_fixed']

  if not isinstance(s, str):
    return s, err, fix
    
  if s == 'нрзб':
    return nrzb, err, fix

  if 'нрзб' in s and nrzb not in s:
    err = True

  for v in nrzb_replacement_dict:
    s = s.replace(v, nrzb_replacement_dict[v])

  if 'нрзб' in s and nrzb not in s:
    fix = False

  return s, err, fix

In [ ]:
for col in df_cols:
  replace_log_res = df.apply(lambda row: replace_log(row, col), axis=1)
  df[col] = replace_log_res.apply(lambda t: t[0])
  df[f'{col}_error'] = replace_log_res.apply(lambda t: t[1])
  df[f'{col}_fixed'] = replace_log_res.apply(lambda t: t[2])

Где осталось нрзб в нестандартизированном виде (залогировано, но исправлено)

In [ ]:
for col in df_cols:
  values = df[col].to_list()
  for v in values:
    if not isinstance(v, str):
      continue
    if nrzb not in v and 'нрзб' in v:
      print(v)

Дорогая Леля. Сейчас получив письмо твое и Симы от 14 нояб. [ноября]. Спасибо за поздравление. Живем по прежнему; здоровы. Сейчас поеду в Стрельну.  21 го возможно там соберется целая компания. Рисую теперь редко. Недавно получила [получил] в подарок от Кузнецова один из его рисунков углем. Продолжай рисовать. Твоя [твой] [нрзб. подпись]
Дорогие Екатерина Степановна, Дима и Зойка! Хочу в двадцатых числах прокатиться до Вас, но в то же время боюсь как бы вас не задержать, быть может вы тоже куда нибудь собираетесь поехать. Напишите пожалуйста когда лучше избрать время. Надя [нрзб.?фамилия]
{г. Одесса, 13 августа 1910 г.} Поздравляю и крепко целую дорогую Наташеньку. Лизочка. Дорогая Наташа, целую и поздраляю тебя с днем рождения. Поцелуй маму, папу, Митю, Раю, Леночку, Колю и Мишеньку. Кланяюсь т. Тане. Будь здорова. Еще раз целую, твой дядя [нрзб *Костя? *Коля?]
30 июня. Милые Варя и папуля! Вот уже с 14-го июня я в Симеизи. Лька, наверное, уже писала Вам об этом. Устроился не плохо. С

In [ ]:
df = pd.concat([df], axis=1)

## Номер открытки + ссылки
* заполнение обязательно
* только числа / строки


In [ ]:
cols = ['Номер открытки', 'Pic_url_1', 'Pic_url_2', 'Resize_url_1','Resize_url_2']
for col in cols:
  if (df[col] == ots).sum() > 0:
    print(col)

In [ ]:
df[['Номер открытки', 'Pic_url_1', 'Pic_url_2', 'Resize_url_1','Resize_url_2']].dtypes

Номер открытки     int64
Pic_url_1         object
Pic_url_2         object
Resize_url_1      object
Resize_url_2      object
dtype: object

## Язык текста открытки
* название языка
* “непонятно”, "иностранный язык" - выгружаем, но логируем


In [ ]:
col = 'Язык текста открытки'

In [ ]:
df[col].value_counts()

русский             17587
английский             33
украинский             29
польский               21
французский            19
немецкий               17
болгарский              5
латышский               3
эстонский               2
белорусский             2
эсперанто               1
чешский                 1
иностранный язык        1
шведский                1
непонятно               1
чувашский               1
финский                 1
сербский                1
татарский               1
Name: Язык текста открытки, dtype: int64

Логирование ошибок

In [ ]:
df[f'{col}_error'] = df.apply(
    lambda row: True if row[col] in ('непонятно', 'иностранный язык') else row[f'{col}_error'],
    axis=1
    )
df[f'{col}_error'].value_counts()

False    17725
True         2
Name: Язык текста открытки_error, dtype: int64

## Перевод текста открытки
* удаляем [отсутствует] (если перевод не требуется, поле оставляется пустым)

In [ ]:
col = 'Перевод текста открытки (если на иностранном языке)'

In [ ]:
df[col] = df[col].apply(lambda s: '' if s == ots else s)

Проверка

In [ ]:
df[col].value_counts()

## Вид орфографии
Допускается:
* Современная
* Дореволюционная
* Латиница/современная
* Латиница/дореволюционная
* Латиница 


In [ ]:
col = 'Вид орфографии'

In [ ]:
df[col].value_counts()

Дореволюционная (ъ,ѣ и т.д.)    9217
Современная                     8334
Латиница/дореволюционная          64
Латиница                          47
Латиница/современная              33
[отсутствует]                     14
Дореволюционная                   10
дореволюционная                    6
современная                        1
0                                  1
Name: Вид орфографии, dtype: int64

In [ ]:
orthography_replacement_dict = {
    'Дореволюционная (ъ,ѣ и т.д.)': 'Дореволюционная',
    'дореволюционная': 'Дореволюционная',
    'современная': 'Современная',
    '0': ots
}

In [ ]:
df[col] = df[col].replace(orthography_replacement_dict)

In [ ]:
df[col].value_counts()

Дореволюционная             9233
Современная                 8335
Латиница/дореволюционная      64
Латиница                      47
Латиница/современная          33
[отсутствует]                 15
Name: Вид орфографии, dtype: int64

## Индекс получателя
* только число 
* или [отсутствует]

In [ ]:
col = 'Индекс получателя'

In [ ]:
def replace_log(row):
  s = row[col]
  err = row[f'{col}_error']
  fix = row[f'{col}_fixed']

  if s == ots or s == nrzb:
    return s, err, fix
  
  if not s.isnumeric():
    return s, True, False
  
  return s, err, fix

In [ ]:
df[col] = df[col].astype(str)
replace_log_res = df.apply(replace_log, axis=1)
df[col] = replace_log_res.apply(lambda t: t[0])
df[f'{col}_error'] = replace_log_res.apply(lambda t: t[1])
df[f'{col}_fixed'] = replace_log_res.apply(lambda t: t[2])

In [ ]:
df[col].value_counts()

[отсутствует]    15270
622042             145
125008             129
428017              71
603059              70
                 ...  
142410               1
300028               1
348011               1
340005               1
312080               1
Name: Индекс получателя, Length: 707, dtype: int64

Нечисленные индексы (залогированы, но не исправлены)

In [ ]:
df[(~df[f'{col}_fixed']) & (df[f'{col}_error'])][[f'{col}_old', col, f'{col}_error', f'{col}_fixed']]

## Тип изображения
* Допускается:
* Фотография
* Картинка/рисунок
* Другое
* Рисунок отсутствует


In [ ]:
col = 'Тип изображения (фотография, картина, другое)'

In [ ]:
df[col].value_counts()

Картинка/рисунок        9869
Фотография              7519
Рисунок отсутствует      168
Другое                   139
[отсутствует]             12
Картинка                   7
картинка                   2
[?Фотография]              2
[? Фотография]             1
[? Картинка/рисунок]       1
хх.хх.хххх                 1
Name: Тип изображения (фотография, картина, другое), dtype: int64

In [ ]:
pic_ots = 'Рисунок отсутствует'

pic_type_replacement_dict = {
    'Картинка': 'Картинка/рисунок',
    'картинка': 'Картинка/рисунок',
    '[? Картинка/рисунок]': 'Картинка/рисунок',
    'хх.хх.хххх': 'Картинка/рисунок',
    '[?Фотография]': 'Фотография',
    '[? Фотография]': 'Фотография',
    'современная': 'Современная',
    ots: pic_ots,
}

In [ ]:
df[col] = df[col].replace(pic_type_replacement_dict)

Проверка

In [ ]:
df[col].value_counts()

Картинка/рисунок       9881
Фотография             7525
Рисунок отсутствует     181
Другое                  140
Name: Тип изображения (фотография, картина, другое), dtype: int64

## Период 
Допускается:
* дореволюционный
* советский
* постсоветский

In [ ]:
col = 'Период (дореволюционный, советский, постсоветский)'

In [ ]:
df[col].value_counts()

Дореволюционные        9374
Советские              7808
Постсоветские           431
[отсутствует]            84
дореволюционный           7
дореволюционная           5
Советская                 2
Дореволюционный           2
Российская Империя        1
советский                 1
Гражданская война         1
[непонятно]               1
Советский                 1
Доревалюционный           1
[?Дореволюционные]        1
[? Дореволюционные]       1
Name: Период (дореволюционный, советский, постсоветский), dtype: int64

In [ ]:
period_replacement_dict = {
    'дореволюционный': 'Дореволюционные',
    'дореволюционная': 'Дореволюционные',
    'Дореволюционный': 'Дореволюционные',
    'Российская Империя': 'Дореволюционные',
    'Советская': 'Советские',
    'советский': 'Советские',
    'Советский': 'Советские',
    'Доревалюционный ': 'Дореволюционные',
    'Доревалюционный': 'Дореволюционные',
    '[?Дореволюционные]': 'Дореволюционные',
    '[? Дореволюционные]': 'Дореволюционные',
    'Гражданская война': 'Советские',
    '[непонятно]': 'Советские'
}

In [ ]:
df[col] = df[col].replace(period_replacement_dict)

Проверка

In [ ]:
df[col].value_counts()

Дореволюционные    9397
Советские          7814
Постсоветские       432
[отсутствует]        84
Name: Период (дореволюционный, советский, постсоветский), dtype: int64

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(df[df[col] == '[непонятно]'][['Pic_url_1', 'Pic_url_2']])

## Страна (откуда)

In [ ]:
col = 'Страна (откуда)'

Смотрим существующие страны

In [ ]:
pd.Series(df[col].unique()).to_csv('countries_from.csv', index=False)

In [ ]:
df[col].value_counts()

Российская Империя             6557
СССР                           5603
[отсутствует]                  4384
Россия                          329
Германия                        206
                               ... 
Португалия                        1
Российская империя                1
[Отсутствует]                     1
Ямайка                            1
Южно-Африканская Республика       1
Name: Страна (откуда), Length: 89, dtype: int64

In [ ]:
countries_replacement = {
    'Российская империя': 'Российская Империя',
    'Российская имерия': 'Российская Империя',
    'Российская имеперия': 'Российская Империя',
    '[?Российская Империя]': 'Российская Империя',
    'Российска империя': 'Российская Империя',
    'Виленская губерния, Дисненский уезд, г. Дисна': 'Российская Империя',
    'Советская Россия': 'Россия',
    'ссср': 'СССР',
    'Российская Республика': 'Россия',
    '[?РСФСР]': 'СССР',
    'РСФСР, г. Ленинград ': 'СССР',
    'CCCР': 'СССР',
    '[? СССР]': 'СССР',
    '[?Чехословакия]': 'Чехословакия',
    '[непонятно]': 'СССР'
}

In [ ]:
df[col] = df[col].replace(countries_replacement)

In [ ]:
with pd.option_context('display.max_colwidth', None):
  display(df[df[col] == 'Российская Республика'][['Pic_url_1', 'Pic_url_2']])

Проверка

In [ ]:
df[col].value_counts()

Российская Империя             6620
СССР                           5610
[отсутствует]                  4462
Россия                          332
Германия                        206
                               ... 
Германская империя                1
Ямайка                            1
Греция                            1
Монако                            1
Южно-Африканская Республика       1
Name: Страна (откуда), Length: 65, dtype: int64

Выгружаем результат

In [ ]:
pd.Series(df[col].unique()).to_csv('countries_from_fix.csv', index=False)

## Страна (куда)

In [ ]:
col = 'Страна (куда)'

In [ ]:
pd.Series(df[col].unique()).to_csv('countries_to.csv', index=False)

In [ ]:
df[col] = df[col].replace(countries_replacement)

Проверка

In [ ]:
df[col].value_counts()

## Теги

In [ ]:
tag_cols = [f'Тег_{i}' for i in range(1, 8)]

Допустимые

In [ ]:
tags_allowed = [
  '',
 'Открытка - благодарность',
 'Открытка-приглашение',
 'Репродукции картин',
 'Изображения животных',
 'Дети на картинке',
 'Изображения кораблей и пароходов',
 'Поздравление на картинке',
 'Шлю привет',
 'Житейское',
 'Новости',
 'Поцелуи из писем',
 'С фронта',
 'Поездка',
 'Просьба',
 'Письма от семьи',
 'Учеба',
 'Благодарность в тексте',
 'В пути',
 'Адрес',
 'Деньги',
 'Отдых',
 'Погода',
 'Дурные вести',
 'Военная служба',
 'Полевая почта',
 'Письма маме',
 'Приглашение в тексте',
 'Письма от мамы',
 'Встреча',
 'Рассказ об искусстве',
 'Рассказ о животных',
 'Загадочная открытка',
 'Письма от отца',
 'Написано детьми',
 'Написано о детях',
 'Письма с парохода',
 'Социальное',
 'Иностранный язык',
 'На фронт',
 'Поздравление в тексте',
 'Деловая переписка',
 'Совет',
 'На память',
 'Простите, что долго не писал',
 'Позже напишу подробнее',
 'Письма отцу',
 'Письма семье',
 'Любовное',
 'Почему долго не писал?',
 'Военное',
 'Новый год',
 'Восьмое марта',
 'Первое мая',
 'День победы',
 'Октябрьская революция',
 'Прочие советские праздники',
 'Пасха',
 'Рождество',
 'Прочие религиозные праздники',
 'День рождения',
 'День свадьбы',
 'Прочие личные праздники',
 'Прочие праздники',
 'Фронтовая',
 'Из путешествия',
 'Именины',
 '1 сентября',
 'День космонавтики',
 'Написано иностранцем по-русски',
 '23 февраля',
 'День учителя'
 ]

Смотрим вариативность

In [ ]:
tags_variations = []
for col in cols:
  tags_variations += list(df[col].unique())
tags_variations = list(set(tags_variations))
tags_variations = [t for t in tags_variations if t not in tags_allowed]
tags_variations

Составляем словарь замены

In [ ]:
tags_replacement_dict = {
 '[? житейское]': 'Житейское',
 '2022-09-01 00:00:00': '1 сентября',
 '2022-05-01 00:00:00': 'Первое мая',
 '[?репродукции картин]': 'Репродукции картин',
 '[? благодарность в тексте]': 'Благодарность в тексте',
 '[? просьба]': 'Просьба',
 '[отсутствует]': '',
 'Учёба': 'Учеба',
 'Др': 'День рождения',
 'День ангела': 'Именины',
 '[?любовное]': 'Любовное',
 'Новы': 'Новый год',
 '2023-09-01 00:00:00': '1 сентября',
 '[?шлю привет]': 'Шлю привет',
 '[? социальное]': 'Социальное',
 'Дет': '',  # написано детям, но не детьми и не о детях
 'Письма': '',  # все нужные "письмовые" теги там уже есть
 'Сердечный привет': 'Шлю привет',
 'Долго не пишите': 'Почему долго не писал?',
 'Уведомление о приезде': 'В пути',
 'От всей души': 'Поздравление в тексте',
 'День защитников отечества': '23 февраля',
 }

In [ ]:
for col in tag_cols:
  df[col] = df[col].apply(lambda x: x.capitalize()).replace(tags_replacement_dict)
  df[f'{col}_fixed'] = df.apply(
    lambda row: False if row[col] not in tags_allowed else row[f'{col}_fixed'],
    axis=1
  )

Проверка

In [ ]:
tags_variations = []
for col in tag_cols:
  tags_variations += list(df[col].unique())
tags_variations = list(set(tags_variations))
tags_variations = [t for t in tags_variations if t not in tags_allowed]

print('\n'.join(tags_variations))

[? новый год * рождество]
Пасха, день рождения
День рождения, новый год


In [ ]:
col = tag_cols[2]
with pd.option_context('display.max_colwidth', None):
  display(df[df[col] == 'День защитника отечества'][['Pic_url_1', 'Pic_url_2']])

,Pic_url_1,Pic_url_2


## Даты

In [ ]:
date_cols = [
    'Дата написания текста (если автор указал явно, не штемпели)',
    'Дата открытки со штемпеля (место отправления)',
    'Дата открытки со штемпеля (место, куда открытка прибыла)',
    'Дата печати открытки'
    ]

Словарь для исправления отдельных случаев

In [ ]:
date_variations = [
    '12.18.1944',
    '14 сентября 1914 года',
    '1914',
    '6/6 19 авг[уста] г.Пенза 11 ч[асов] дня',
    'Петербург',
    'xx.xx.xx',
    '26.12.07',
    '08.xx.xx',
    '16.09.07',
    '26.01.10',
    'отсутствует*30.12.1986',
    '23.12.xx',
    '29.07.12',
    '8 марта - международных женский день С праздником',
    'хх.хх.1993/04.12.1992',
    '26.12.1911/ 08.01.1912',
    'xx.xx.xxxxx',
    '20.06.1909\n21.06.1909',
    '18*19.08.1917',
    '30*31.12.1972',
    'Июль 1912 г.',
 'Его высокоблагородие',
 '14.12.88',
 '17.0[?6].81',
 'хх.хх.1993/04.12.1992',
 '1961 год',
 '23.01.80',
 'März',
 'господин',
 '[?23].01.[?80]',
 '30/IX-76г.',
 '06.0[2*3].1972',
 '11.2x.1962',
 '30.31.1901',
'26.12.1911/ 08.01.1912',
'18/31.07.1914',
]

date_replacement_dict = {
    '12.18.1944': '18.12.1944',
    '14 сентября 1914 года': '14.09.1914',
    '1914': 'xx.xx.1914',
    '6/6 19 авг[уста] г.Пенза 11 ч[асов] дня': '19.08.xxxx',
    'Петербург': ots,
    'xx.xx.xx': 'xx.xx.xxxx',
    'xx.xx.xxxxx': 'xx.xx.xxxx',
    '8 марта - международных женский день С праздником': ots,
    # 'хх.хх.1993/04.12.1992',
    # '26.12.1911/ 08.01.1912'
    # '20.06.1909\n21.06.1909'
    'Июль 1912 г.': 'xx.07.1912',
#  'Его высокоблагородие',
#  'März',
#  'господин',
 '30/IX-76г.': '30.09.1976',
#  '06.0[2*3].1972'
'11.2x.1962': '2x.11.1962',
# '30.31.1901',
# '26.12.1911/ 08.01.1912',
# '18/31.07.1914',
 '[отсутствует*30.12.1986]': '30.12.1986',
}

Форматирование даты

In [ ]:
def format_date(s, period):
  # убираем г, г. или год в конце
  s = re.sub('г(\.|од)?$', '', s)

  # убираем [], {}, ?, !, ^, лишние точки
  s = re.sub('\[', '', s)
  s = re.sub('\]', '', s)
  s = re.sub('\{', '', s)
  s = re.sub('\}', '', s)
  s = re.sub('\?', '', s)
  s = re.sub('!', '', s)
  s = re.sub('^', '', s)

  s = s.lstrip().rstrip()
  s = re.sub('\.\.', '.', s)
  s = re.sub('\.$', '', s)

  # заменяем x кириллицей на латиницу
  s = re.sub('х', 'x', s)

  # если указан только год
  m = re.match('[0-9]{4}', s)
  if m is not None and m.group(0) == s:
    s = f'xx.xx.{s}'
    return s

  # приводим к формату dd.mm.yyyy
  # если в формате yyyy-mm-dd 00:00:00
  m = re.match(timestamp_pattern, s)
  if m is not None and m.group(0) == s:
    s = datetime.strptime(s, '%Y-%m-%d %H:%M:%S').strftime("%d.%m.%Y")
    return s

  # если день / месяц указан одной цифрой, год указан двумя цифрами
  try:
    day, month, year = s.split('.')
    day = day.lstrip().rstrip()
    month = month.lstrip().rstrip()
    year = year.lstrip().rstrip()
    
    if len(day) == 1:
      if day == 'x':
        day = 'xx'
      else:
        day = f'0{day}'
    
    if len(month) == 1:
      if month == 'x':
        month = 'xx'
      else:
        month = f'0{month}'
    
    if len(year) == 2:
      if 'x' in year:
        year = f'xx{year}'
      else:
        year = get_year_from_period(year, period)
    
    s = f'{day}.{month}.{year}'
  
  except:
    pass

  return s

Верный формат даты

In [ ]:
# xx.xx.xxxx
# date_pattern = '[0-3x][0-9x]\.[0-1x][0-9x]\.[0-9x]{4}(\s?\-\s?[0-3x][0-9x]\.[0-1x][0-9x]\.[0-9x]{4})?'
date_pattern = '[0-3x][0-9x]\.[0-1x][0-9x]\.[0-9x]{4}'

# yyyy-mm-dd 00:00:00
timestamp_pattern = '[0-9]{4}-[0-1][0-9]-[0-3][0-9] 00:00:00'

Доопределение года по периоду
* 1869 - 1917 - дореволюционные
* 1918 - 1991 - советские
* 1992 - … - постсоветские


In [ ]:
def get_year_from_period(year, period):
  year = int(year)
  if period == 'Дореволюционные':
    if 69 <= year <= 99:
      year = f'18{year}'
    else:
      year = f'19{year}'
  if period in ('Советские', 'Постсоветские'):
      year = f'19{year}'
  return year

In [ ]:
get_year_from_period(61, 'Советские')

'1961'

Главная функция замены и логирования

In [ ]:
def replace_log(row):
  s = row[col]
  err = row[f'{col}_error']
  fix = row[f'{col}_fixed']
  period = row['Период (дореволюционный, советский, постсоветский)']
  
  if s == ots or s == nrzb:
    return s, err, fix

  if s in date_replacement_dict:
    return date_replacement_dict[s], True, fix

  m = re.match(date_pattern, s)
  if m is not None and m.group(0) == s:
    return s, err, fix

  s = format_date(s, period)
  m = re.match(date_pattern, s)
  fix = False
  if m is not None and m.group(0) == s:
    fix = True
  return s, True, fix

In [ ]:
for col in date_cols:
  replace_log_res = df.apply(replace_log, axis=1)
  df[col] = replace_log_res.apply(lambda t: t[0])
  df[f'{col}_error'] = replace_log_res.apply(lambda t: t[1])
  df[f'{col}_fixed'] = replace_log_res.apply(lambda t: t[2])

Проверка

In [ ]:
col = date_cols[3]
df[df[f'{col}_error']][[f'{col}_old', col, f'{col}_error']].sample(10)

,Дата печати открытки_old,Дата печати открытки,Дата печати открытки_error
1495,1973-05-14 00:00:00,14.05.1973,True
2368,1976-09-06 00:00:00,06.09.1976,True
5940,1978-05-25 00:00:00,25.05.1978,True
1280,1971-12-20 00:00:00,20.12.1971,True
1318,1973,xx.xx.1973,True
2764,1985-03-11 00:00:00,11.03.1985,True
1279,1963,xx.xx.1963,True
1169,2013-03-26 00:00:00,26.03.2013,True
6002,1991-05-22 00:00:00,22.05.1991,True
330,1983-12-16 00:00:00,16.12.1983,True


Залогировано, но не исправлено

In [ ]:
not_fixed = pd.concat([
  df[~df[f'{col}_fixed']][
      ['Pic_url_1', 'Pic_url_2', f'{col}_old', col, f'{col}_error', f'{col}_fixed']
      ].rename(columns={
          f'{col}_old': 'old',
          col: 'new',
          f'{col}_error': 'error',
          f'{col}_fixed': 'fixed'
      }
  )
  for col in date_cols
])

with pd.option_context('display.max_colwidth', None):
  display(not_fixed)

# Логирование ошибок
* ошибкой считается, если старое и новое значение ячейки не совпадают
* или уже зарегистрирована ошибка

* если старое и новое значение ячейки совпадают с точностью до пробелов в начале и в конце: 
  - если новое значение = [отсутствует] или [нрзб], считаем, что значения не совпадают
  - иначе, считаем, что значения совпадают

In [ ]:
def log_errors(row, col):
  old_stripped = row[f'{col}_old']
  if isinstance(old_stripped, str):
    old_stripped = old_stripped.lstrip().rstrip()
  new = row[col]
  error = row[f'{col}_error']

  if error:
    return True

  if old_stripped != new:
    return True
  
  if old_stripped == new and (new == ots or new == nrzb):
    return True

  return False

In [ ]:
for col in df_cols:
  df[f'{col}_error'] = df.apply(lambda row: log_errors(row, col), axis=1)

Готово к выгрузке в корпус, если ошибки не было или она была исправлена

In [ ]:
def get_dump_status(row):
  for col in df_cols:
    if not row[f'{col}_fixed']:
      return False
  return True

In [ ]:
df['dump_ready'] = df.apply(get_dump_status, axis=1)
df['dump_ready'].value_counts()

True     17653
False       74
Name: dump_ready, dtype: int64

Выгрузка логов, где ошибка есть, но не исправлена

In [ ]:
def get_error_cols(row):
  res = []
  for col in df_cols:
    if row[f'{col}_error'] and not row[f'{col}_fixed']:
      res.append(col)
  return '\n'.join(res)

In [ ]:
df_for_corr = df[~df['dump_ready']]
df_for_corr['error_cols'] = df_for_corr.apply(get_error_cols, axis=1)
df_for_corr['error_cols'].head()

11                                       Имя отправителя
37                       Текст открытки\nИмя отправителя
65     Текст открытки\nИмя отправителя\nАдрес получателя
723                      Текст открытки\nИмя отправителя
860                                 Дата печати открытки
Name: error_cols, dtype: object

In [ ]:
with open('2023-06-05-corpus_logs_for_correction.txt', 'w') as corr_logs:
  for ind, row in df_for_corr.iterrows():
    corr_logs.write(
        f"{row['Номер открытки']}\n{row['error_cols']}\n\n"
        )

In [ ]:
errors_dict = dict()

with open('2023-06-05-corpus_logs_for_correction.txt', 'r') as fin:
  for s in fin:
    s = s.strip()
    if s == '':
      continue
    if s.isnumeric():
      postcard_id = int(s)
      errors_dict[postcard_id] = []
    else:
      errors_dict[postcard_id].append(s)

errors_dict

In [8]:
errors_df = []
for postcard_id in errors_dict:
  for col in errors_dict[postcard_id]:
    errors_df.append([postcard_id, col])
errors_df[:10]

[[78, 'Имя отправителя'],
 [179, 'Текст открытки'],
 [179, 'Имя отправителя'],
 [231, 'Текст открытки'],
 [231, 'Имя отправителя'],
 [231, 'Адрес получателя'],
 [1573, 'Текст открытки'],
 [1573, 'Имя отправителя'],
 [1971, 'Дата печати открытки'],
 [2202, 'Текст открытки']]

In [11]:
errors_df = pd.DataFrame(errors_df, columns=['Номер открытки', 'Столбец с ошибкой'])
errors_df.head()

,Номер открытки,Столбец с ошибкой
0,78,Имя отправителя
1,179,Текст открытки
2,179,Имя отправителя
3,231,Текст открытки
4,231,Имя отправителя


In [14]:
errors_df['База'] = errors_df['Номер открытки'].apply(
    lambda x: 1 if x < 100000
    else 2 if x < 200000
    else 3 if x < 215837
    else 4 if x < 225880
    else 'xxx'
)

In [15]:
errors_df['База'].value_counts()

3    36
4    18
2    15
1    11
Name: База, dtype: int64

In [16]:
errors_df['Статус'] = 'Не исправлено'
errors_df = errors_df[['База', 'Номер открытки', 'Столбец с ошибкой', 'Статус']]

In [17]:
errors_df.to_csv('2023-06-05-corpus_logs.csv', index=False)

Проверка логики: нет такого, что ошибки нет и ошибка не исправлена

In [ ]:
for ind, row in df.iterrows():
  for col in df_cols:
    if col in ['Номер открытки', 'Pic_url_1', 'Pic_url_2', 'Resize_url_1', 'Resize_url_2']:
      continue
    if not row[f'{col}_error'] and not row[f'{col}_fixed']:
      print(row['Номер открытки'])
      print(col)
      print(row[f'{col}_old'], '\t', row[col])
      print()

# Выгрузка корпуса

## Корпус - original

* убираем ссылки на сжатые изображения
* убираем технические столбцы
* оставляем только открытки, готовые к выгрузке

In [ ]:
df_cols = ['Номер открытки', 'Pic_url_1', 'Pic_url_2', 'Resize_url_1', 'Resize_url_2'] + df_cols

In [ ]:
corpus = df[df['dump_ready']][df_cols]
corpus = corpus.rename(
    columns={'Pic_url_1': 'Лицевая сторона - ссылка', 
             'Pic_url_2': 'Оборотная сторона - ссылка'}
             ).drop(['Resize_url_1', 'Resize_url_2'], axis=1)

In [ ]:
corpus.shape

(17653, 33)

In [ ]:
corpus.head()

In [ ]:
cur_date = datetime.today().strftime('%Y.%m.%d')
filename = f'[{cur_date}] Пишу тебе. Корпус (база 1-4) - original.csv'
filename

'[2023.05.06] Пишу тебе. Корпус (база 1-4) - original.csv'

In [ ]:
corpus.to_csv(filename, index=False)

## Корпус - версия наружу

* убираем ссылки на исходники изображений 
* убираем технические столбцы
* оставляем только открытки, готовые к выгрузке

In [ ]:
corpus = df[df['dump_ready']][df_cols]
corpus = corpus.rename(
    columns={'Resize_url_1': 'Лицевая сторона - ссылка', 
             'Resize_url_2': 'Оборотная сторона - ссылка'}
             ).drop(['Pic_url_1', 'Pic_url_2'], axis=1)

In [ ]:
corpus.shape

(17653, 33)

In [ ]:
corpus.head()

In [ ]:
filename = f'[{cur_date}] Пишу тебе. Корпус (база 1-4).csv'
filename

'[2023.05.06] Пишу тебе. Корпус (база 1-4).csv'

In [ ]:
corpus.to_csv(filename, index=False)